In [1]:
import ee
ee.Initialize()

In [15]:
from geetools import ui, tools, algorithms, Satellite

In [3]:
Map = ui.Map()
Map.show()

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…

TEST

In [4]:
i = ee.Image('LANDSAT/LC8_L1T_TOA_FMASK/LC81780592016030LGN00').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B9', 'B10', 'fmask'],
['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'thermal', 'fmask']);

In [5]:
ic = algorithms.Landsat.brdf_correct(i)

In [6]:
p = ee.Geometry.Point([22.0227, 1.4801])
Map.centerObject(p, zoom=10)

In [7]:
ival = 0.10098563879728317
icval = 0.10012929931321499

In [8]:
ival_here = tools.image.get_value(i, p, 30, 'client')['blue']

In [9]:
ival_here == ival

True

In [10]:
icval_here = tools.image.get_value(ic, p, 30, 'client')['blue']

In [11]:
icval_here == icval

True

In [12]:
vis = {'bands':['nir', 'swir1','red'], 'min':0, 'max': 0.5}

In [13]:
Map.addLayer(i, vis, 'original')

In [14]:
Map.addLayer(ic, vis, 'corrected')

EXAMPLE WITH `geetools.tools.Satellite`

In [16]:
col = Satellite('LANDSAT_8_SR')

In [17]:
image = ee.Image(col.collection.filterMetadata(col.cloud_cover['property'], 'less_than', 10).filterBounds(p).first())

In [18]:
vis = {'bands':[col.bands['nir'], col.bands['swir1'], col.bands['red']], 'min':0, 'max':5000}

In [19]:
Map.addLayer(image, vis, 'original using Satellite')

In [22]:
corrected = algorithms.Landsat.brdf_correct(image, satellite='LANDSAT_8_SR')

In [23]:
Map.addLayer(corrected, vis, 'corrected using Satellite')